In [2]:
import pandas as pd

#%store df save df from another .py file
#%store -r df read df from another .py file

df = pd.read_excel('139329 Fette tableting data_FILTERED.xlsx')
#df = pd.read_csv('6MonthTest.csv', sep=";")
#df.reset_index(drop=True) #sets the index to one column
#Timestamp column should be named as "timestamp". Upper of lowercases do not play any role

In [13]:
#str(df['Timestamp'].dtypes)

'datetime64[ns]'

In [ ]:
import pandas as pd

#%store df save df from another .py file
#%store -r df read df from another .py file

df = pd.read_excel('139329 Fette tableting data_FILTERED.xlsx')
#df = pd.read_csv('6MonthTest.csv', sep=";")
#df.reset_index(drop=True) #sets the index to one column
#Timestamp column should be named as "timestamp". Upper of lowercases do not play any role


#################################################################

import numpy as np
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc
import plotly.graph_objects as go
import csv
import scipy.stats as stats
import seaborn as sns
from datetime import datetime as dt

cols = list(df.columns)
columns = pd.Series(cols)
app = dash.Dash(__name__, external_stylesheets= [dbc.themes.BOOTSTRAP])
app.config.suppress_callback_exceptions = True
collapse = html.Div([
    dbc.Button("Open collapse", id="collapse-button", className="mb-3", color="primary"),
    dbc.Collapse(dbc.Card(dbc.CardBody("Hidden")),
                id="collapse"),
])

navbar = dbc.NavbarSimple(
    children=[
        dbc.NavItem(dbc.NavLink("Histogram", href="/histogram"), id="page-1-link"),
        dbc.NavItem(dbc.NavLink("Lineplot", href="/lineplot"), id="page-2-link"),
        dbc.NavItem(dbc.NavLink("Scatterplot", href="/scatterplot"), id="page-3-link"),
        dbc.NavItem(dbc.NavLink("Multiplot", href="/multiplot"), id="page-4-link"),
        dbc.NavItem(dbc.NavLink("Pairplot", href="/pairplot"), id="page-5-link"),
        dbc.NavItem(dbc.NavLink("Boxplot", href="/boxplot"), id="page-6-link"),
        dbc.NavItem(dbc.NavLink("Jointplot", href="/distplot"), id="page-7-link"),
        dbc.NavItem(dbc.NavLink("Correlations", href="/corrmat"), id="page-8-link"),
    ],
    brand="Plotter",
    brand_href="/",
    color="green",
    dark=True,
)

app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    html.Div(id='page-content'),
])

index_page = html.Div([
    navbar,
    html.H1(children='Welcome to Plotter'),
])

page_1_layout = html.Div([navbar,
                          dbc.Row([dbc.Col([dcc.Dropdown(id='x-axis',options=[{'label': i, 'value': i} for i in columns.unique()],
                                                        placeholder = "SELECT TAG",multi=True, optionHeight = 20),
                                           dcc.Dropdown(id='histogram-group',options=[{'label': i, 'value': i} for i in columns.unique()],
                                                        placeholder = "SELECT GROUP",multi=False, optionHeight = 20)]),
                                   dbc.Col([dbc.Col([html.Div("Bin amount:"), dcc.Input(id="binsize", type="number", value=100)], style={'display': 'inline-block'}),
                                           dbc.Col([dcc.RadioItems(id = "histnorm",options=[{'label': 'Count', 'value': ''},
                                                                                   #{'label': 'Probability', 'value': 'probability'},
                                                                                   {'label': 'Probability density', 'value': 'probability density'}], value = 'probability density')])
                                           ])
                                  ]),
                          dbc.Row(html.Div(children=html.Div(id='graphs'), className='col'))])

@app.callback(
    Output(component_id='graphs', component_property='children'),
    [Input(component_id='x-axis', component_property='value'),
    Input(component_id='binsize', component_property='value'),
    Input(component_id='histogram-group', component_property='value'),
    Input(component_id='histnorm', component_property='value')])


def make_graphs(columns, nbins, histcolumn, histnorm):
    graphs = []
    if columns is None:
        return None
    else:
        if histcolumn is None:
            traces = []
            for col in columns:
                min_value=df[col].min()
                max_value=df[col].max()
                size = (max_value-min_value)/nbins
                plotdata = list(df[col])
                hist = go.Histogram(
                    x = list(plotdata),
                    name = "Hist: {}".format(col),
                    histnorm = histnorm,
                    opacity=0.5,
                    xbins=dict(start=min_value, end=max_value, size=size
                ))
                if histnorm == 'probability density':
                    nparam_density = stats.kde.gaussian_kde(df[col].values.ravel())
                    x = np.linspace(min_value, max_value, nbins)
                    nparam_density = nparam_density(x)
                    trace = go.Scattergl(x=list(x), y=list(nparam_density), name="KDE: {}".format(col))
                    traces.append(hist)
                    traces.append(trace)
                else:
                    traces.append(hist)
            graphs.append(html.Div(dcc.Graph(id=col, figure={'data': traces,
                                                             'layout': go.Layout(margin={'l':50,'r':50,'t':45,'b':40},
                                                                                 barmode = 'overlay', height = 750,title='{}'.format(col))})))
            return graphs
        else:
            traces = []
            graphs = []
            for col in columns:
                colgraphs = None
                min_value=df[col].min()
                max_value=df[col].max()
                size = (max_value-min_value)/nbins
                uniquekeys = list(df[histcolumn].unique())
                uniquekeys = sorted(uniquekeys)
                i = 0
                for key in uniquekeys:
                    data_group = df[df[histcolumn]==key]
                    plotdata = list(data_group[col])
                    hist = go.Histogram(
                        x = plotdata,
                        name = "{}: {}".format(col, key),
                        histnorm = histnorm,
                        opacity=0.5,
                        xbins=dict(start=min_value, end=max_value, size=size)
                    )
                    traces.append(hist)
                    if histnorm == 'probability density':
                        #data_test = df[df[col]==key]
                        #nparam_density = stats.kde.gaussian_kde(df[col].values.ravel())
                        nparam_density = stats.kde.gaussian_kde(data_group[col].values.ravel())
                        x = np.linspace(min_value, max_value, nbins)
                        nparam_density = nparam_density(x)
                        probdens = go.Scattergl(x=list(x), y=list(nparam_density), name="KDE - {}: {}".format(col, key))
                        traces.append(probdens)
                    else:
                        pass
                    i = i + 1
                colgraph = html.Div(dcc.Graph(id=col,
            
            figure={'data': traces,'layout' : go.Layout(margin={'l':50,'r':50,'t':45,'b':40}, barmode = 'overlay', height = 750,
                                                                hovermode = 'closest',
                                                                title='{}'.format(str(columns)))}))
    
            graphs.append(colgraph)
            return graphs
        

        
# page_2_layout = html.Div([navbar,
#                           dcc.Dropdown(id='Timestamp-column', options=[{'label': i, 'value': i} for i in columns.unique()], placeholder = "SELECT DATETIME COLUMN"),
#                           dcc.DatePickerRange(id='datepicker2', number_of_months_shown = 3, with_portal = True, with_full_screen_portal = True),
#                           dcc.Dropdown(
#                               id='page-2-x-axis',
#                               options=[{'label': i, 'value': i} for i in columns.unique()],
#                               placeholder = "SELECT X AXIS", optionHeight = 20),
#                           dcc.Dropdown(
#                               id='page-2-y-axis', options=[{'label': i, 'value': i} for i in columns.unique()],
#                               placeholder = 'SELECT Y AXIS', optionHeight = 20,
#                               multi = True),
#                           html.Div(children=html.Div(id='page-2-graphs'), className='col'),
#                          ])

page_2_layout = html.Div([navbar,
                          dcc.Dropdown(id='page-2-x-axis',options=[{'label': i, 'value': i} for i in columns.unique()],
                              placeholder = "SELECT X AXIS", optionHeight = 20,value = df.columns[0]),
                          dcc.Dropdown(id='page-2-y-axis',options=[{'label': i, 'value': i} for i in columns.unique()],
                                       placeholder = 'SELECT Y AXIS', optionHeight = 20, value = df.columns[1]),
                          dcc.Dropdown(id='page-2-color', options=[{'label': i, 'value': i} for i in columns.unique()],
                                       value = 'GROUP BY'),
                          html.Div(id='page-2-graphs')])

@app.callback(
    Output(component_id='page-2-graphs', component_property='children'),
    [Input(component_id='page-2-y-axis', component_property='value'),
    Input(component_id='page-2-x-axis', component_property='value'),
    Input(component_id='page-2-color', component_property='value')])

def make_graphs2(ycolumn, xcolumn, group):
    #unique = df[group].nunique()
    if group == 'GROUP BY':
        graphs = []
        xdata = list(df[xcolumn])
        ydata = list(df[ycolumn])
        #color = df['TimeStamp'].to_numpy() ++++ marker=dict(color=color, colorscale='Viridis', showscale = True)
        trace0 = go.Scattergl(x=xdata, y=ydata, mode='markers', showlegend=False, name = str(ydata))
        data = [trace0]
        layout = go.Layout(xaxis={'title': '{}'.format(xcolumn)}, yaxis={'title': '{}'.format(ycolumn)}, height = 800, hovermode='closest')
        graphs.append(html.Div(dcc.Graph(id="lineplot1", figure={'data': data, 'layout': layout})))
        
        return graphs
    
    else:
        graphs = []
        traces = []
        xdata = list(df[xcolumn])
        ydata = list(df[ycolumn])
        uniquekeys = list(df[group].unique())
        uniquekeys = sorted(uniquekeys)
        i = 0
        for groupkey in uniquekeys:
            dff = df[df[group]==groupkey]
            xdata = list(dff[xcolumn])
            ydata = list(dff[ycolumn])
            traces.append(go.Scattergl(x=xdata, y=ydata, mode='lines', showlegend=True, name = str(groupkey)))
            i = i+1
        
        layout = go.Layout(xaxis={'title': '{}'.format(xcolumn)}, yaxis={'title': '{}'.format(ycolumn)}, height = 800, hovermode='closest')
        
        graphs.append(html.Div(dcc.Graph(id="lineplot1", figure={'data': traces, 'layout': layout})))
        
        return graphs

page_3_layout = html.Div([navbar,
                          dcc.Dropdown(id='page-3-x-axis', options=[{'label': i, 'value': i} for i in columns.unique()],
                              placeholder = "SELECT X AXIS", optionHeight = 20, value = df.columns[0]),
                          dcc.Dropdown(id='page-3-y-axis', options=[{'label': i, 'value': i} for i in columns.unique()],
                              placeholder = 'SELECT Y AXIS', optionHeight = 20, value = df.columns[1]),
                          dcc.Dropdown(id='page-3-color', options=[{'label': i, 'value': i} for i in columns.unique()],
                              placeholder = 'SELECT TAG FOR COLORING'),
                          html.Div(id='page-3-graph', style={"height" : 800})])

@app.callback(
    Output(component_id='page-3-graph', component_property='children'),
    [Input(component_id='page-3-y-axis', component_property='value'),
    Input(component_id='page-3-x-axis', component_property='value'),
    Input(component_id='page-3-color', component_property='value')])

def make_graphs3(ycolumn, xcolumn, group):
    if group == None:
        graphs = []
        xdata = list(df[xcolumn])
        ydata = list(df[ycolumn])
        trace0 = go.Scattergl(x=xdata, y=ydata, mode='markers', showlegend=True, name = str(ycolumn))
        data = [trace0]
        layout = go.Layout(xaxis={'title': '{}'.format(xcolumn)}, yaxis={'title': '{}'.format(ycolumn)}, height = 800, hovermode='closest')
        graphs.append(html.Div(dcc.Graph(id="scatterplot", figure={'data': data, 'layout': layout})))
        
        return graphs
    
    elif group != None:
        datatype = str(df[group].dtypes)
        if "datetime" in datatype:
            traces = []
            graphs = []
            xdata = list(df[xcolumn])
            ydata = list(df[ycolumn])
            listrange = np.arange(0, len(xdata))
            testlist = np.random.randn(len(xdata))
            traces.append(go.Scattergl(x=xdata, y=ydata, mode='markers', showlegend=False,
                                       marker=dict(color=listrange, colorscale='Viridis', showscale=True)))
            layout = go.Layout(xaxis={'title': '{}'.format(xcolumn)}, yaxis={'title': '{}'.format(ycolumn)}, height = 800, hovermode='closest')
            graphs.append(html.Div(dcc.Graph(id="scatterplot1", figure={'data': traces, 'layout': layout})))
            return graphs
        
        else:
            traces = []
            graphs = []
            xdata = list(df[xcolumn])
            ydata = list(df[ycolumn])
            uniquekeys = list(df[group].unique())
            uniquekeys = sorted(uniquekeys)
            i = 0
            for groupkey in uniquekeys:
                dff = df[df[group]==groupkey]
                xdata = list(dff[xcolumn])
                ydata = list(dff[ycolumn])
                traces.append(go.Scattergl(x=xdata, y=ydata, mode='markers', showlegend=True, name = str(groupkey)))
                i = i+1
            layout = go.Layout(xaxis={'title': '{}'.format(xcolumn)}, yaxis={'title': '{}'.format(ycolumn)}, height = 800, hovermode='closest')
            graphs.append(html.Div(dcc.Graph(id="scatterplot1", figure={'data': traces, 'layout': layout})))
            return graphs
    else:
        return None
    

page_4_layout = html.Div([navbar,
                          dcc.Dropdown(
                              id='page-4-x-axis',
                              options=[
                                  {'label': i, 'value': i} for i in columns.unique()
                              ],
                              placeholder = "SELECT X AXIS", optionHeight = 20,
                              value = df.columns[0],
                          ),
                          dcc.Dropdown(
                              id='page-4-y-axis',
                              options=[
                                  {'label': i, 'value': i} for i in columns.unique()
                              ],
                              placeholder = 'SELECT Y AXIS', optionHeight = 20,
                              multi = True
                          ),
                          html.Div([
                          dcc.Graph(id='page-4-graph')
                          ], style={"height" : 800})])

@app.callback(
    Output(component_id='page-4-graph', component_property='figure'),
    [Input(component_id='page-4-x-axis', component_property='value'),
    Input(component_id='page-4-y-axis', component_property='value')])

def make_graphs4(xcolumn, ycolumn):
    data = []
    xdata = list(df[xcolumn])
    lines = 0
    domain = [0,1]
    position3=0
    position4=1
    if ycolumn != None:
        
        for col in ycolumn:
            lines = lines+1
            if lines == 1:
                ydata = list(df[col])
                yline = go.Scattergl(x = xdata, y=ydata, mode='lines', name = col)
                data.append(yline)
            elif lines == 2:
                ydata = list(df[col])
                yline = go.Scattergl(x = xdata, y=ydata, mode='lines', name = col, yaxis='y2')
                data.append(yline)
            elif lines == 3:
                domain = [0.025,0.975]
                position3 = 0.01
                ydata = list(df[col])
                yline = go.Scattergl(x = xdata, y=ydata, mode='lines', name = col, yaxis='y3')
                data.append(yline)
            elif lines == 4:
                domain = [0.025,0.975]
                position3 = 0.01
                position4 = 0.99
                ydata = list(df[col])
                yline = go.Scattergl(x = xdata, y=ydata, mode='lines', name = col, yaxis='y4')
                data.append(yline)
            elif lines == 5:
                position3 = 0.03
                position4 = 0.975
                domain = [0.05,0.95]
                ydata = list(df[col])
                yline = go.Scattergl(x = xdata, y=ydata, mode='lines', name = col, yaxis='y5')
                data.append(yline)
            elif lines == 6:
                position3 = 0.03
                position4 = 0.975
                domain = [0.05,0.95]
                ydata = list(df[col])
                yline = go.Scattergl(x = xdata, y=ydata, mode='lines', name = col, yaxis='y6')
                data.append(yline)
            else:
                domain = [0.15,0.85]
                ydata = list(df[col])
                yline = go.Scattergl(x = xdata, y=ydata, mode='lines', name = col, yaxis='y7')
                data.append(yline)

    return {
            'data': data,
            'layout': go.Layout(xaxis={'title': xcolumn, 'domain': domain}, height = 800,
                                yaxis=dict(tickfont=dict(color='blue')),
                                yaxis2=dict(overlaying='y', side='right', tickfont=dict(color="orange")),
                                yaxis3=dict(overlaying='y', side='left', tickfont=dict(color="green"), anchor="free", position=position3),
                                yaxis4=dict(overlaying='y', side='right', tickfont=dict(color="red"), anchor="free", position=position4),
                                yaxis5=dict(overlaying='y', side='left', tickfont=dict(color="blueviolet"), anchor="free", position=0.01),
                                yaxis6=dict(overlaying='y', side='right', tickfont=dict(color="brown"), anchor="free", position=0.99))
        }

page_5_layout = html.Div([navbar,
                          dcc.Dropdown(id='pairplot-columns', options=[{'label': i, 'value': i} for i in columns.unique()],
                                       placeholder = "SELECT TAG", optionHeight = 20,
                                       multi=True),
                          html.Div(id="splomplot")])

@app.callback(
    Output(component_id='splomplot', component_property='children'),
    [Input(component_id='pairplot-columns', component_property='value')])

def make_pairplot(columns):
    if columns != None:
        graphs = []
        dimensions = []
        for col in columns:
            col_data = df[col]
            label = col
            dimension = dict(label=label, values=col_data)
            dimensions.append(dimension)
        data = [go.Splom(dimensions=dimensions, text=None, marker=dict(color='red', size=7, showscale=False))]
        layout = go.Layout(height = 800, hovermode = 'closest')
        graphs.append(html.Div(dcc.Graph(id="pairplot", figure={'data': data, 'layout': layout})))

        return graphs
            
    return None
        
      


page_6_layout = html.Div([navbar,
                          dcc.Dropdown(
                              id='boxploty',
                              options=[
                                  {'label': i, 'value': i} for i in columns.unique()
                              ],
                              placeholder = "SELECT COLUMNS FOR BOXPLOTS",
                              multi=True, optionHeight = 20
                          ),
                          dcc.Dropdown(
                              id='boxplotx',
                              options=[
                                  {'label': i, 'value': i} for i in columns.unique()
                              ],
                              placeholder = "SELECT X AXIS TAG",
                              multi=False, optionHeight = 20
                          ),
                          html.Div(id="boxplot")])

@app.callback(
    Output(component_id='boxplot', component_property='children'),
    [Input(component_id='boxploty', component_property='value'),
    Input(component_id='boxplotx', component_property='value')])

def make_boxplot(columny, groupcolumn):
    if columny != None:
        if groupcolumn != None:
            graphs = []
            traces = []
            groupcolumndata = pd.Categorical(df[groupcolumn])
#             uniquekeys = list(groupcolumndata.unique())
#             uniquekeys = sorted(uniquekeys)
            xdata = list(groupcolumndata)
            for col in columny:
                ydata = list(df[col])
                data = go.Box(y=ydata, x=groupcolumndata, name = str(col))
                traces.append(data)
                layout = go.Layout(height = 800, hovermode = 'closest', boxmode = 'group')
                graphs.append(html.Div(dcc.Graph(id="boxplot1", figure={'data': traces,'layout' : layout})))
            
            return graphs

            #return "{}".format(type(groupcolumndata))            

        else:
            graphs = []
            traces = []
            for col in columny:
                ydata = list(df[col])
                data = go.Box(y=ydata, name = str(col))
                traces.append(data)
                layout = go.Layout(height = 800, hovermode = 'closest')
                graphs.append(html.Div(dcc.Graph(id="boxplot1", figure={'data': traces,'layout' : layout})))

            return graphs
    return None

page_7_layout = html.Div([navbar,
                          html.Div([html.Div('x-axis bin amount:', style={'display': 'inline-block'}), 
                                    dcc.Input(id="xbinsize-distplot", type="number", value=100)], style={'display': 'inline-block'}),
                          html.Div([html.Div('y-axis bin amount:', style={'display': 'inline-block'}), 
                                    dcc.Input(id="ybinsize-distplot", type="number", value=100)]
                                   ,style={'display': 'inline-block'}),
                          html.Div([dcc.RadioItems(id="kde-on-off", options=[{'label': 'Show KDE plot', 'value': 'probability density'},
                                                                             {'label': 'Hide KDE plot', 'value': ''}],value='')]
                                   ,style={'display': 'inline-block'}),
                          dcc.Dropdown(id='xaxiscolumn', options=[{'label': i, 'value': i} for i in columns.unique()],
                              placeholder = "SELECT X AXIS TAG", value = df[df.columns[1]], optionHeight = 20, multi=False),
                          dcc.Dropdown(id='yaxiscolumn', options=[{'label': i, 'value': i} for i in columns.unique()],
                              placeholder = "SELECT Y AXIS TAG", optionHeight = 20, multi=False),
                          dcc.Dropdown(id='groupcolumn', options=[{'label': i, 'value': i} for i in columns.unique()], placeholder = "COLOR BY TAG",),
                          html.Div(id='distplots')])


@app.callback(
    Output(component_id='distplots', component_property='children'),
    [Input(component_id='xaxiscolumn', component_property='value'),
     Input(component_id='yaxiscolumn', component_property='value'),
     Input(component_id='groupcolumn', component_property='value'),
     Input(component_id='xbinsize-distplot', component_property='value'),
     Input(component_id='ybinsize-distplot', component_property='value'),
    Input(component_id='kde-on-off', component_property='value')])

def make_distplots(xaxis, yaxis, groupcolumn, xbins, ybins, kde):
    graphs = []
    colorvalues = ["blue", "red", "green", "black", "yellow", "brown", "orange", "black", "violet", "aqua", "darkgreen", "magenta", "silver", "sienna", 
                   "tan", "turquoise", "olive", "orangered", "lime", "navy", "palegreen", "darkkhaki", "sienna", "slategrey"]
    if xaxis != None:
        if yaxis != None:
            if groupcolumn == None:
                min_x_value=df[xaxis].min()
                max_x_value=df[xaxis].max()
                min_y_value=df[yaxis].min()
                max_y_value=df[yaxis].max()
                x_size = (max_x_value-min_x_value)/xbins
                y_size = (max_y_value-min_y_value)/ybins

                xdata = list(df[xaxis])
                ydata = list(df[yaxis])
                trace1 = go.Scattergl(x = xdata, y = ydata, marker = dict(color='blue', opacity=0.5), mode = 'markers', name = 'Scatterplot')
                trace2 = go.Histogram(x=xdata, name='{}'.format(xaxis), yaxis='y2', marker = dict(color='blue'), histnorm = 'probability density', xbins=dict(start=min_x_value, end=max_x_value, size=x_size))
                trace3 = go.Histogram(y=ydata, name='{}'.format(yaxis), xaxis='x2', marker = dict(color='blue'), histnorm = 'probability density', ybins=dict(start=min_y_value, end=max_y_value, size=y_size))
                
#                 if kde == 'probability density':
#                     nparam_density = stats.kde.gaussian_kde(df[col].values.ravel())
#                     x = np.linspace(min_value, max_value, nbins)
#                     nparam_density = nparam_density(x)
#                     trace = go.Scattergl(x=list(x), y=list(nparam_density), name="KDE: {}".format(col))
#                     traces.append(hist)
#                     traces.append(trace)
#                 else:
#                     traces.append(hist)
                
                data = [trace1, trace2, trace3]
                layout = go.Layout(height = 900, width = 1600,hovermode = 'closest', 
                                   yaxis2=dict(domain=[0.75, 1], showgrid=True, zeroline=True),
                                   xaxis2=dict(domain=[0.85, 1], showgrid=True, zeroline=True),
                                   yaxis=dict(domain=[0, 0.75], showgrid=True, zeroline=True),
                                   xaxis=dict(domain=[0, 0.85], showgrid=True, zeroline=True))

                graphs.append(html.Div(dcc.Graph(id="distplot",
                                                 figure={'data': data,'layout' : layout})))

                return graphs
            else:
                min_x_value=df[xaxis].min()
                max_x_value=df[xaxis].max()
                min_y_value=df[yaxis].min()
                max_y_value=df[yaxis].max()
                x_size = (max_x_value-min_x_value)/xbins
                y_size = (max_y_value-min_y_value)/ybins
                uniquekeys = list(df[groupcolumn].unique())
                uniquekeys = sorted(uniquekeys)
                traces = []
                i = 0
                for key in uniquekeys:
                    dff = df[df[groupcolumn] == key]
                    xdata = list(dff[xaxis])
                    ydata = list(dff[yaxis])
                    trace = go.Scattergl(x = xdata, y = ydata, name = "{}: SCATTER".format(key), mode = 'markers', marker=dict(color = colorvalues[i], opacity = 0.5))
                    traces.append(trace)
                    histy = go.Histogram(x = xdata, yaxis='y2', name = "{}: Y - HIST".format(key), histnorm = 'probability density',
                                         xbins=dict(start=min_x_value, end=max_x_value, size=x_size), opacity = 0.5, marker_color = colorvalues[i])
                    histx = go.Histogram(y = ydata, xaxis='x2', name = "{}: X - HIST".format(key), histnorm = 'probability density',
                                         ybins=dict(start=min_y_value, end=max_y_value, size=y_size), opacity = 0.5, marker_color = colorvalues[i])
                    traces.append(histy)
                    traces.append(histx)
                    i = i + 1
                layout = go.Layout(height = 900, width = 1800, hovermode = 'closest', barmode = 'overlay',
                                   yaxis2=dict(domain=[0.75, 1], showgrid=True, zeroline=True),
                                   xaxis2=dict(domain=[0.85, 1], showgrid=True, zeroline=True),
                                   yaxis=dict(domain=[0, 0.75], showgrid=True, zeroline=True),
                                   xaxis=dict(domain=[0, 0.85], showgrid=True, zeroline=True))

                graphs.append(html.Div(dcc.Graph(id=groupcolumn,
                                                 figure={'data': traces,'layout' : layout})))

                return graphs
        return None
    return None
            
page_8_layout = html.Div([navbar,
                          dcc.RadioItems(id = "correlationmethod",options=[{'label': 'Pearson', 'value': 'pearson'},
                                                                 {'label': 'Spearman', 'value': 'spearman'},
                                                                 {'label': 'Kendall', 'value': 'kendall'}],
                                                   value = 'pearson', style={'display': 'inline-block'}),
                          dcc.Dropdown(id='correlation-columns', optionHeight = 20,
                              options=[{'label': i, 'value': i} for i in columns.unique()],
                              placeholder = "SELECT TAG", multi=True),
                          html.Div(id='correlationplot')])

@app.callback(
    Output(component_id='correlationplot', component_property='children'),
    [Input(component_id='correlation-columns', component_property='value'),
    Input(component_id='correlationmethod', component_property='value')])

def make_correlationplot(columns, method):
    if columns == None:
        return None
    else:
        annotations = []
        columns_list = list(columns)
        dff = df[columns_list]
        df_corr = dff.corr(method=method)
        #df_corr = df_corr.sort_values(columns_list, ascending=False)
        values = df_corr.values
        values = values.round(2)
        for n, row in enumerate(values):
                for m, val in enumerate(row):
                    annotations.append(go.layout.Annotation(text=str(values[n][m]), x=columns[m], y=columns[n],
                                                            xref='x1', yref='y1', showarrow=False, font=dict(color="#ffffff", size=12)))
        graafi = html.Div(dcc.Graph(
                        id='heatmap',
                        figure={
                            "data": [go.Heatmap(z=values,
                                                x=columns_list,
                                                y=columns_list,
                                                #colorscale = colorscale
                                               )
                            ],
                            "layout": {
                                'annotations': annotations,
                                'yaxis': {'autorange': 'reversed', 'automargin': True},
                                'xaxis': {'side': 'top', 'automargin': True},
                                'autosize': True,
                                'margin': {'t':'50', 'l':'200', 'r':'0', 'b':'0'},
                                'width': 1200,
                                'height': 900
                            },
                        },
                    ),
                className = 'row'
            )

        return graafi
    
@app.callback(Output('page-content', 'children'), [Input('url', 'pathname')])

def display_page(pathname):
    if pathname == '/histogram':
        return page_1_layout
    elif pathname == '/lineplot':
        return page_2_layout
    elif pathname == '/scatterplot':
        return page_3_layout
    elif pathname == '/multiplot':
        return page_4_layout
    elif pathname == '/pairplot':
        return page_5_layout
    elif pathname == '/boxplot':
        return page_6_layout
    elif pathname == '/distplot':
        return page_7_layout
    elif pathname == '/corrmat':
        return page_8_layout
    else:
        return index_page

@app.callback(Output('page-1-link', 'active'), [Input('url', 'pathname')])
def set_page_1_active(pathname):
    return pathname == '/histogram'

@app.callback(Output('page-2-link', 'active'), [Input('url', 'pathname')])
def set_page_2_active(pathname):
    return pathname == '/lineplot'

@app.callback(Output('page-3-link', 'active'), [Input('url', 'pathname')])
def set_page_3_active(pathname):
    return pathname == '/scatterplot'

@app.callback(Output('page-4-link', 'active'), [Input('url', 'pathname')])
def set_page_4_active(pathname):
    return pathname == '/multiplot'

@app.callback(Output('page-5-link', 'active'), [Input('url', 'pathname')])
def set_page_5_active(pathname):
    return pathname == '/pairplot'

@app.callback(Output('page-6-link', 'active'), [Input('url', 'pathname')])
def set_page_6_active(pathname):
    return pathname == '/boxplot'

@app.callback(Output('page-7-link', 'active'), [Input('url', 'pathname')])
def set_page_7_active(pathname):
    return pathname == '/distplot'

@app.callback(Output('page-8-link', 'active'), [Input('url', 'pathname')])
def set_page_8_active(pathname):
    return pathname == '/corrmat'

if __name__ == '__main__':
    app.run_server(port=8060)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8060/ (Press CTRL+C to quit)
127.0.0.1 - - [20/Nov/2019 10:52:30] "GET /histogram HTTP/1.1" 200 -
127.0.0.1 - - [20/Nov/2019 10:52:30] "GET /_dash-component-suites/dash_renderer/react@16.8.6.min.js?v=1.0.0&m=1565787319 HTTP/1.1" 200 -
127.0.0.1 - - [20/Nov/2019 10:52:30] "GET /_dash-component-suites/dash_renderer/prop-types@15.7.2.min.js?v=1.0.0&m=1565787319 HTTP/1.1" 200 -
127.0.0.1 - - [20/Nov/2019 10:52:30] "GET /_dash-component-suites/dash_renderer/react-dom@16.8.6.min.js?v=1.0.0&m=1565787319 HTTP/1.1" 200 -
127.0.0.1 - - [20/Nov/2019 10:52:30] "GET /_dash-component-suites/dash_html_components/dash_html_components.min.js?v=1.0.0&m=1565787318 HTTP/1.1" 200 -
127.0.0.1 - - [20/Nov/2019 10:52:30] "GET /_dash-component-suites/dash_core_components/highlight.pack.js?v=1.1.1&m=1565787318 HTTP/1.1" 200 -
127.0.0.1 - - [20/Nov/2019 10:52:31] "GET /_dash-component-suites/dash_bootstrap_components/_components/dash_bootstrap_components.min.js?v=0.7.0&m=1566382750